# 감정 + 개인화 하이브리드 음식 추천 (날씨 반영/반복 섭취 고도화)

이 노트북은 기본 데모를 확장하여,
- 날씨(기온/상태) 반영
- 반복 섭취 패널티 고도화(최근성/빈도/카테고리 다양성)
를 포함한 최종 추천을 제공합니다.

## 0. 환경 준비
Colab에서 실행 시 아래 패키지 설치 셀을 필요에 따라 사용하세요. (기본 데모는 추가 설치 없이 동작)

In [ ]:
# 선택: OpenWeatherMap API 사용 시 필요한 라이브러리
# !pip install requests python-dotenv

import datetime
import math
from typing import List, Dict, Any, Tuple


## 1. 감정 분석 (데모: 규칙/키워드 기반)
실 서비스에서는 한국어 사전학습 모델(KLUE/KoBERT 등) 또는 임베딩+분류기 권장.

In [ ]:
def simple_emotion_classifier(text: str) -> Dict[str, Any]:
    t = text.lower()
    if any(k in t for k in ['스트레스', 'stress', '짜증', '빡침']):
        return {'emotion': 'stress', 'score': 0.9}
    if any(k in t for k in ['슬프', 'sad', '우울', '눈물']):
        return {'emotion': 'sad', 'score': 0.85}
    if any(k in t for k in ['피곤', 'tired', '졸려', '지침']):
        return {'emotion': 'tired', 'score': 0.8}
    if any(k in t for k in ['외로', 'lonely', '쓸쓸', '허전']):
        return {'emotion': 'lonely', 'score': 0.8}
    return {'emotion': 'neutral', 'score': 0.6}

# 예시
simple_emotion_classifier('오늘 스트레스가 너무 쌓여서 화가 난다')


## 2. 감정 → 음식 1차 매핑
실제 서비스에서는 Firestore/Supabase 등 외부 저장소 권장.

In [ ]:
emotion_food_map: Dict[str, List[str]] = {
    'stress': ['짬뽕', '삼겹살', '떡볶이', '마라탕', '불고기', '카레'],
    'sad': ['미역국', '닭죽', '칼국수', '된장찌개', '솥밥'],
    'tired': ['닭가슴살 샐러드', '연어덮밥', '두부구이', '콩나물국밥', '비빔밥'],
    'lonely': ['치킨', '파스타', '피자', '햄버거', '크림리조또'],
    'neutral': ['김치찌개', '비빔국수', '순두부찌개', '샌드위치']
}

def get_candidates_by_emotion(emotion: str, score: float, top_k: int = 5) -> List[str]:
    pool = emotion_food_map.get(emotion, emotion_food_map['neutral'])
    if score >= 0.8:
        return pool[:top_k]
    return pool

get_candidates_by_emotion('stress', 0.9)


## 3. 음식 카테고리 분류(다양성 계산용)
- 카테고리별로 최근 섭취 빈도/최근성 기반 패널티를 계산하기 위해 사용합니다.

In [ ]:
food_category: Dict[str, str] = {
    # 면/국물
    '짬뽕': 'spicy_soup', '칼국수': 'noodle_soup', '콩나물국밥': 'soup', '된장찌개': 'soup', '미역국': 'soup',
    # 고기/단백질
    '삼겹살': 'meat', '불고기': 'meat', '닭가슴살 샐러드': 'salad', '두부구이': 'protein',
    # 분식/자극
    '떡볶이': 'k_snack', '마라탕': 'spicy_soup', '김치찌개': 'soup',
    # 한식/덮밥/밥류
    '솥밥': 'rice', '비빔밥': 'rice', '연어덮밥': 'rice',
    # 양식/패스트푸드
    '파스타': 'western', '피자': 'western', '햄버거': 'western', '크림리조또': 'western', '샌드위치': 'western',
    # 기타
    '순두부찌개': 'soup', '비빔국수': 'noodle', '카레': 'rice'
}

def get_category(food: str) -> str:
    return food_category.get(food, 'other')

get_category('삼겹살'), get_category('짬뽕'), get_category('연어덮밥')


## 4. 유저 프로필 + 최근 섭취 로그(타임스탬프 포함)
- 실제에선 Firestore: `users/{uid}/recent_food_logs`에 `food`, `timestamp`, `category` 저장 권장
- 여기선 데모 데이터로 시뮬레이션합니다.

In [ ]:
now = datetime.datetime.now()
hours = lambda h: datetime.timedelta(hours=h)

user_profile = {
    'uid': 'demo_user',
    'likes': ['삼겹살', '비빔밥', '파스타'],
    'dislikes': ['마라탕'],
    'sensitive_spicy': True,
    # 최근 섭취 로그: (음식, 시각) — 실제에선 UNIX epoch 또는 ISO8601 권장
    'recent_logs': [
        {'food': '비빔밥', 'time': now - hours(10)},
        {'food': '삼겹살', 'time': now - hours(28)},
        {'food': '짬뽕', 'time': now - hours(50)},
        {'food': '파스타', 'time': now - hours(73)},
    ]
}

# 최근 로그에 카테고리 필드 추가(보기 편의)
for log in user_profile['recent_logs']:
    log['category'] = get_category(log['food'])

user_profile['recent_logs']


## 5. 날씨 입력(수동/선택적 API)
- 기본: 수동 입력으로 기온(°C), 상태(`clear`, `rain`, `snow`, `hot`, `cold` 등)을 지정
- 선택: OpenWeatherMap API를 통해 현재 날씨를 받아올 수 있는 예시 코드 제공(주석)

In [ ]:
# 수동 입력 예시
weather = {
    'temp_c': 3.0,       # 현재 기온(섭씨)
    'status': 'cold'     # 'hot'|'warm'|'mild'|'cold'|'rain'|'snow' 등
}
weather


In [ ]:
# 선택: OpenWeatherMap API 연동 예시
# import os, requests
# from dotenv import load_dotenv
#
# def fetch_weather_by_city(city: str, api_key: str) -> Dict[str, Any]:
#     url = f'https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric'
#     r = requests.get(url, timeout=10)
#     r.raise_for_status()
#     data = r.json()
#     temp_c = data['main']['temp']
#     cond = data['weather'][0]['main'].lower()  # e.g., 'rain', 'snow', 'clear'
#     # 간단 매핑
#     status = 'rain' if 'rain' in cond else ('snow' if 'snow' in cond else ('clear' if 'clear' in cond else 'mild'))
#     return {'temp_c': temp_c, 'status': status}
#
# # 사용 예시
# # load_dotenv()
# # api_key = os.getenv('OPENWEATHER_API_KEY')
# # weather = fetch_weather_by_city('Seoul', api_key)
# # weather


## 6. 날씨 적합도 함수
- 추울 때: 따뜻한 국물/칼칼한 국물 가점
- 더울 때: 냉한/가벼운/샐러드류 가점
- 비/눈: 따뜻한 국물/면류 가점
점수 범위는 0~0.1 내에서 가감(가중치 0.1에 해당).

In [ ]:
cold_favor = {'soup', 'spicy_soup', 'noodle_soup'}
hot_favor = {'salad', 'noodle', 'western'}  # 가벼운/차가운/산뜻한 쪽 예시
rain_snow_favor = {'soup', 'noodle_soup', 'spicy_soup'}

def weather_suitability(food: str, weather: Dict[str, Any]) -> float:
    cat = get_category(food)
    temp = weather.get('temp_c', 20)
    status = weather.get('status', 'mild')
    s = 0.0
    # 온도 기반
    if temp <= 5 and cat in cold_favor:
        s += 0.06
    if temp >= 28 and cat in hot_favor:
        s += 0.06
    # 상태 기반
    if status in ('rain', 'snow') and cat in rain_snow_favor:
        s += 0.04
    return min(max(s, -0.1), 0.1)

weather_suitability('콩나물국밥', weather), weather_suitability('닭가슴살 샐러드', weather)


## 7. 반복 섭취 패널티 고도화
- 최근성 가중: 가까운 시점에 먹은 음식일수록 더 큰 패널티
- 빈도 패널티: 최근 윈도우에서 많이 먹을수록 패널티
- 카테고리 다양성: 같은 카테고리 반복 시 패널티, 다른 카테고리면 약간 보너스
패널티/보너스 합은 대략 -0.2~+0.05 범위를 권장(가중치 0.2 근사).

In [ ]:
def time_decay(hours_diff: float, half_life: float = 24.0) -> float:
    """
    시간 경과에 따른 감쇠 가중치(0~1). half_life 시간 지나면 0.5로 감소.
    최근일수록 1에 가깝고, 오래될수록 0에 가까움.
    """
    return 0.5 ** (hours_diff / half_life)

def recent_repetition_penalty(food: str, logs: List[Dict[str, Any]], now: datetime.datetime, window_hours: float = 72.0) -> float:
    # window 내 등장한 동일 음식에 대해 시간가중 합을 계산하고 -0.2 범위를 넘지 않도록 제한
    penalty = 0.0
    for log in logs:
        h = (now - log['time']).total_seconds() / 3600.0
        if h <= window_hours and log['food'] == food:
            penalty += 0.2 * time_decay(h)  # 최근일수록 크게
    return -min(penalty, 0.2)  # 최대 -0.2

def frequency_penalty(food: str, logs: List[Dict[str, Any]], now: datetime.datetime, window_hours: float = 168.0) -> float:
    # 1주(168h) 내 동일 음식 빈도에 따른 소규모 패널티 (-0.1 한도)
    cnt = 0
    for log in logs:
        h = (now - log['time']).total_seconds() / 3600.0
        if h <= window_hours and log['food'] == food:
            cnt += 1
    return -min(0.05 * cnt, 0.1)

def category_diversity_adjust(food: str, logs: List[Dict[str, Any]], now: datetime.datetime, window_hours: float = 72.0) -> float:
    # 최근 window 내 마지막 N개(예: 5개)의 카테고리 분포를 보고 동일 카테고리 반복 시 -0.05, 다르면 +0.02
    cat = get_category(food)
    recent = [log for log in logs if (now - log['time']).total_seconds()/3600.0 <= window_hours]
    recent_sorted = sorted(recent, key=lambda x: x['time'], reverse=True)[:5]
    same = sum(1 for l in recent_sorted if l.get('category') == cat)
    if same >= 2:
        return -0.05
    if same == 0:
        return 0.02
    return 0.0

# 테스트
recent_repetition_penalty('비빔밥', user_profile['recent_logs'], now), frequency_penalty('삼겹살', user_profile['recent_logs'], now), category_diversity_adjust('파스타', user_profile['recent_logs'], now)


## 8. 개인화 점수 함수(확장)
질문에서 제시한 가중치 구조를 유지하면서 날씨·반복 로직을 반영합니다.
- 감정 매칭 0.3
- 사용자 선호 0.3
- 시간대 0.1
- 건강/체질 0.1
- 최근 섭취 패널티 0.2 (최근성/빈도/카테고리 다양성 포함)
- 날씨 적합도는 시간대/건강 범주 외 보정치로 0~0.1 내에서 가감

In [ ]:
# 시간대/건강/자극 카테고리 예시
morning_foods = {'샌드위치', '닭가슴살 샐러드', '죽', '미역국'}
heavy_foods = {'삼겹살', '치킨', '피자'}
spicy_foods = {'짬뽕', '떡볶이', '마라탕', '김치찌개'}

def personalized_score(food: str, user_profile: Dict[str, Any], emotion: str, emotion_score: float, weather: Dict[str, Any]) -> float:
    score = 0.0

    # 1) 감정 점수
    if food in emotion_food_map.get(emotion, []):
        score += 0.3

    # 2) 선호도
    if food in user_profile.get('likes', []):
        score += 0.3
    if food in user_profile.get('dislikes', []):
        score -= 0.3

    # 3) 시간대
    hour = datetime.datetime.now().hour
    if hour < 11 and food in morning_foods:
        score += 0.1
    if hour > 20 and food in heavy_foods:
        score -= 0.1

    # 4) 건강/체질
    if user_profile.get('sensitive_spicy') and food in spicy_foods:
        score -= 0.1

    # 5) 최근 섭취(고도화)
    logs = user_profile.get('recent_logs', [])
    score += recent_repetition_penalty(food, logs, now)  # 최대 -0.2
    score += frequency_penalty(food, logs, now)          # 최대 -0.1
    score += category_diversity_adjust(food, logs, now)  # -0.05 ~ +0.02

    # 6) 날씨 적합도 보정(0~0.1)
    score += weather_suitability(food, weather)

    return round(float(score), 4)


## 9. 최종 추천 함수

In [ ]:
def recommend_final(user_profile: Dict[str, Any], emotion: str, emotion_score: float, weather: Dict[str, Any], topn: int = 3) -> List[Tuple[str, float]]:
    candidates = get_candidates_by_emotion(emotion, emotion_score, top_k=5)
    scored = []
    for food in candidates:
        s = personalized_score(food, user_profile, emotion, emotion_score, weather)
        scored.append((food, s))
    scored.sort(key=lambda x: x[1], reverse=True)
    return scored[:topn]

emo = simple_emotion_classifier('오늘 스트레스가 많고 추워서 몸이 으슬으슬해')
result = recommend_final(user_profile, emo['emotion'], emo['score'], weather, topn=3)
{
    'emotion': emo['emotion'],
    'weather': weather,
    'top3': [ {'food': f, 'score': s} for f, s in result ]
}


## 10. 조정 포인트
- 날씨 가중치: `weather_suitability`의 값(0.06/0.04 등)을 조정해 영향력 튜닝
- 최근성/빈도: `time_decay` half-life, 윈도우 크기(72h/168h), 상한(-0.2/-0.1) 조정
- 다양성: `category_diversity_adjust`의 보정치(-0.05/+0.02) 조절
- 카테고리 정의: `food_category`를 서비스 도메인에 맞게 정교화
- Firestore 연동: 최근 로그/선호/건강 정보를 실시간으로 반영
- AB 테스트: 사용자군 별 가중치/규칙을 나눠 실험 후 최적값 도출